In [2]:
import pandas as pd

test_df = pd.read_csv('Data/test.csv')
test_df.json = test_df.json.apply(lambda x: eval(x))
test_df

,json,text,label
0,"{'departure_city': 'Верцелли', 'arrival_city':...",Я хочу купить билет из Верцелли в Успенку с 1...,самолет
1,"{'departure_city': 'Тускалуса', 'arrival_city'...",Я хочу купить билет в Тускалусе в день 16-го ...,самолет
2,"{'departure_city': 'Годалминг', 'arrival_city'...",Я хочу купить билет в авиакомпании на рейс из...,самолет
3,"{'departure_city': 'Бодайбо', 'arrival_city': ...","Я хочу купить билет в авиакомпанию, отправляя...",самолет
4,"{'departure_city': 'Старая Кулатка', 'arrival_...",Я живу в Старой Кулатке и хочу купить билет в...,самолет
...,...,...,...
395,"{'city': 'Албена', 'hotel': 'Suite Top em Cont...",Я хотел бы забронировать номер в Suite Top em...,отель
396,"{'city': 'Эуген', 'hotel': 'Holiday Inn Expres...","Здравствуйте, я хотел бы забронировать номер ...",отель
397,"{'city': 'Гурьевск', 'hotel': 'Shoalwater 401 ...",*Я хотел бы забронировать номер в отеле Shoal...,отель
398,"{'city': '', 'hotel': 'Hotel Deepak', 'date': ...",Я хотел бы забронировать номер в Hotel Deepak...,отель


# Merge model

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

model_name = 'mistralai/Mistral-7B-v0.1'
model_dir = 'APP/models/Mistral_NER_merged'

base_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model_to_merge = PeftModel.from_pretrained(base_model, 'NER-mistral_v0.1/checkpoint-200/')
merged_model = model_to_merge.merge_and_unload()
merged_model.save_pretrained(model_dir, safe_serialization=True)

tokenizer.save_pretrained(model_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('APP/models/Mistral_NER_merged/tokenizer_config.json',
 'APP/models/Mistral_NER_merged/special_tokens_map.json',
 'APP/models/Mistral_NER_merged/tokenizer.model',
 'APP/models/Mistral_NER_merged/added_tokens.json',
 'APP/models/Mistral_NER_merged/tokenizer.json')

# Test

In [1]:
from vllm import LLM, SamplingParams
from tqdm import tqdm
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
llm = LLM(model="APP/models/Mistral_NER_merged")

INFO 04-07 17:28:58 llm_engine.py:70] Initializing an LLM engine with config: model='APP/models/Mistral_NER_merged', tokenizer='APP/models/Mistral_NER_merged', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 04-07 17:29:06 llm_engine.py:275] # GPU blocks: 12690, # CPU blocks: 2048
INFO 04-07 17:29:07 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-07 17:29:07 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 04-07 17:29:09 model_runner.py:547] Graph capturing finished in 3 secs.


In [4]:
from utils import *
ner = NER()

In [5]:
sampling_params = SamplingParams(temperature=0, max_tokens=100, stop='}', include_stop_str_in_output=True)
for i in tqdm(range(400)):
    sample = test_df.iloc[i]
    prompt = f"""Ты должен найти в тексте определенные атрибуты и сохранить их в формате JSON. Не используй комментарии.

Используй следующий шаблон:

Текст: сообщение пользователя в чате.
JSON: объект по всем стандартам JSON.{ner.label_mapping[sample.label]['template']}
Начинай!

Текст: {sample.text}
Ответ:"""
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text
    ner.count_errors(sample, generated_text)

100%|██████████| 400/400 [08:26<00:00,  1.27s/it]


In [6]:
ner.score()

Точность для класса "самолет"

	departure_city: 0.86500

	arrival_city: 0.89500

	departure_date: 0.79000

	return_date: 0.81500

Точность для класса "отель"

	city: 0.58500

	hotel: 0.91500

	date: 0.86500

	guests: 0.96000

	days: 0.60000



# Тест классификации

In [7]:
import pandas as pd

df = pd.read_csv('Data/generated_data.csv')

# Zero-shot learning

In [8]:
prediction = []
sampling_params = SamplingParams(temperature=0, max_tokens=30, stop=['отель', 'самолет'], include_stop_str_in_output=True)
for i in tqdm(range(2000)):
    sample = df.iloc[i]
    prompt = f"""Ты должен отнести текст к одному из двух классов: самолет и отель.
    
    Используй следующий шаблон:
    
    Текст: сообщение пользователя в чате
    Класс: одно из двух значений: самолет или отель
    
    Начинай!
    
    Текст: {sample.text}
    Класс: """
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text
    generated_text = ''.join([char for char in generated_text if char.isalpha()])
    prediction.append(generated_text)

100%|██████████| 2000/2000 [02:07<00:00, 15.70it/s]


In [9]:
df['prediction'] = prediction
booking_errors = df[(df['prediction']!=df['label'])&(df['label']=='отель')].shape[0]
ticket_errors = df[(df['prediction']!=df['label'])&(df['label']=='самолет')].shape[0]
print(f'Accuracy booking = {1-booking_errors/1000:.3f}\nAccuracy tickets = {1-ticket_errors/1000:.3f}')

Accuracy booking = 0.995
Accuracy tickets = 0.998


## One-shot learning

In [10]:
prediction_one_shot = []
sampling_params = SamplingParams(temperature=0, max_tokens=30, stop=['отель', 'самолет'], include_stop_str_in_output=True)
for i in tqdm(range(2000)):
    sample = df.iloc[i]
    prompt = f"""Ты должен отнести текст к одному из двух классов: самолет и отель.

    Пример для класса самолет: {df.iloc[0].text}
    Пример для класса отель: {df.iloc[1000].text}
    
    Используй следующий шаблон:
    
    Текст: сообщение пользователя в чате
    Класс: одно из двух значений: самолет или отель
    
    Начинай!
    
    Текст: {sample.text}
    Класс: """
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text
    generated_text = ''.join([char for char in generated_text if char.isalpha()])
    prediction_one_shot.append(generated_text)

100%|██████████| 2000/2000 [02:45<00:00, 12.09it/s]


In [11]:
df['prediction_one_shot'] = prediction_one_shot
booking_errors = df[(df['prediction_one_shot']!=df['label'])&(df['label']=='отель')].shape[0]
ticket_errors = df[(df['prediction_one_shot']!=df['label'])&(df['label']=='самолет')].shape[0]
print(f'Accuracy booking = {1-booking_errors/1000:.3f}\nAccuracy tickets = {1-ticket_errors/1000:.3f}')

Accuracy booking = 1.000
Accuracy tickets = 1.000
